In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import zipfile
import os
import shutil



In [6]:
df = pd.read_excel('/Users/songtengyu/Desktop/7.10上上订单.xlsx')
df.head(40)

,打印时间：,Unnamed: 1,2024-07-10 16:43:59,Unnamed: 3,Unnamed: 4,Unnamed: 5,页码：第1页，共4页,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,采购订单,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,订单日期：2024-07-10,NaN,NaN,NaN,NaN,订单编号：PO240710096,NaN,NaN,NaN,NaN
3,供方,单位名称,江苏上上电缆集团有限公司,NaN,NaN,联系电话,NaN,13179771777,NaN,NaN
4,NaN,单位地址,溧阳市上上路68号,NaN,NaN,传 真,NaN,无,NaN,NaN
5,NaN,联 系 人,李仁武,NaN,NaN,E_mail,NaN,NaN,NaN,NaN
6,需方,单位名称,德州锦城电装股份有限公司,NaN,NaN,联系电话,NaN,NaN,NaN,NaN
7,NaN,单位地址,中国山东省德州市临邑县城区开元西大街富民路东侧,NaN,NaN,传 真,NaN,NaN,NaN,NaN
8,NaN,联 系 人,NaN,NaN,NaN,E_mail,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def parse_order(df):
    df_all_orders = pd.DataFrame()
    # select order rows between
    # NO	产品编号	描述	规格	供方料号	数量	未税单价	未税金额	含税金额	交货日期
    # and
    # <NA>	<NA>	<NA>	合计	<NA>	<NA>	41814.05	<NA>
    start = None
    end = None
    for i in range(len(df)):
        if df.iloc[i, 0] == 'NO':
            start = i
        if df.iloc[i, 5] == '合计':
            end = i
        if start and end:
            df_info = df.iloc[start:end, :]
            df.columns = df_info.iloc[0, :]
            df_info = df_info[1:]
            df_all_orders = pd.concat([df_all_orders, df_info])
            start = None
            end = None
    return df_all_orders


df_parsed = parse_order(df)
df_parsed

,NO,产品编号,描述,规格,供方料号,数量,未税单价,未税金额,含税金额,交货日期
11,1,06-01FLRY-B075O-W,电线类_电线_FLRY-B075O-W,FLRY-B075O-W,NaN,4000,0.5261,2104.4,2377.97,2024-07-20
12,2,06-01FLRY-A035Y-BR,电线类_电线_FLRY-A035Y-BR,FLRY-A035Y-BR,NaN,6500,0.2564,1666.6,1883.26,2024-07-20
13,3,06-01FLRY-A035Y-W,电线类_电线_FLRY-A035Y-W,FLRY-A035Y-W,NaN,6500,0.2564,1666.6,1883.26,2024-07-20
14,4,06-01FLRY-B075B-BR,电线类_电线_FLRY-B075B-BR,FLRY-B075B-BR,NaN,4000,0.5261,2104.4,2377.97,2024-07-20
15,5,06-01FLRY-B075B-L,电线类_电线_FLRY-B075B-L,FLRY-B075B-L,NaN,4000,0.5261,2104.4,2377.97,2024-07-20
16,6,06-01FLRY-B075B-P,电线类_电线_FLRY-B075B-P,FLRY-B075B-P,NaN,12000,0.5261,6313.2,7133.92,2024-07-20
17,7,06-01FLRY-B075B-Y,电线类_电线_FLRY-B075B-Y,FLRY-B075B-Y,NaN,8000,0.5261,4208.8,4755.94,2024-07-20
18,8,06-01FLRY-B075GR-W,电线类_电线_FLRY-B075GR-W,FLRY-B075GR-W,NaN,8000,0.5261,4208.8,4755.94,2024-07-20
19,9,06-01FLRY-B075L-W,电线类_电线_FLRY-B075L-W,FLRY-B075L-W,NaN,8000,0.5261,4208.8,4755.94,2024-07-20
20,10,06-01FLRY-B075P-GR,电线类_电线_FLRY-B075P-GR,FLRY-B075P-GR,NaN,4000,0.5261,2104.4,2377.97,2024-07-20
